### load model

In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
from joblib import dump

model_name = "stock_model_name.keras" # or name.h5
# save scaler: dump(scaler, "scaler.joblib")
scaler_name = "scaler_stock_name.joblib"
# file name of saved features (use json file)
feature_file_name = "features_stock_name.json"
# load features
features = load_features(feature_file_name)
# input test data
input_data = None
model = load_model(model_name)
## scaler must be the that in the model training
scaler = load(scaler_name)

$IFX.DE: possibly delisted; no price data found  (period=10y)


### nomalize input data (here, the training scaler must be used！）
#### # Note: use "transform" instead of "fit_transform"

In [ ]:
## new_features = [['Open', 'High', 'Low', 'Close', 'Volume']]
scaled_data = scaler.transform(features)

### prepare to predict data

In [ ]:
window_size = 60
prepared_data = np.array([scaled_data[-window_size:]])

### predict data

In [ ]:
predict_scaled_data = model.predict(prepared_data)

### invert normalized data

In [ ]:
def _invert_normalized_data(self):
    # 反归一化需要重建完整的多变量矩阵（仅Close列有值，其他列置0）
    dummy_matrix = np.zeros((len(self._Y_predict), self.features.shape[1]))
    dummy_matrix[:, 3] = self._Y_predict.flatten()  # 第4列是Close
    self._Y_pred_actual = self._scaler.inverse_transform(dummy_matrix)[:, 3]
    return dummy_matrix